In [88]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions

In [89]:
year = 2019
paper_

In [106]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)
iclr_link = f"https://iclr.cc/Conferences/{year}/Schedule?type=Poster"
driver.get(iclr_link)
paper_number = len(driver.find_elements_by_class_name("Poster"))
print(paper_number)

502


In [108]:
for paper_idx in range(paper_number):
    driver.get(iclr_link)
    papers = driver.find_elements_by_class_name("Poster")
    paper = papers[poster_idx]
    pdf_link = paper.find_elements_by_class_name(
        "href_PDF")
    if len(pdf_link) == 0:
        continue
    pdf_link = pdf_link[0].get_attribute("href")

    pdf_link = pdf_link.replace("forum", "pdf")
    paper_name = paper.find_element_by_class_name(
        "maincardBody").get_attribute("innerHTML")
    data["pdf_link"] = pdf_link
    data["name"] = paper_name
    paper.click()

    def get_authors():
        main = driver.find_element_by_id('main')
        authors = main.find_elements_by_class_name("glyphicon-user")
        return authors
    paper_link = driver.current_url
    author_number = len(get_authors())

    for author_idx in range(author_number):
        driver.get(paper_link)
        author = get_authors()[author_idx]
        author_link = author.find_element_by_xpath('..')
        author_link.click()
        card = driver.find_element_by_class_name('maincard')
        author_name = card.find_element_by_tag_name(
            'h3').get_attribute("innerHTML")
        author_affiliation = card.find_element_by_tag_name(
            'h4').get_attribute("innerHTML")
        data['authors'].append((author_name, author_affiliation))



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=74.0.3729.131)
  (Driver info: chromedriver=74.0.3729.131 (518a41c1fa7ce1c8bb5e22346e82e42b4d76a96f-refs/branch-heads/3729@{#954}),platform=Linux 5.0.13-arch1-1-ARCH x86_64)
